In [114]:
import pandas as pd
import numpy as np
import sys
import random
from keras.models import Model
from keras.layers import Dense, Input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
sys.path.append("..")

In [34]:
import utils
import game_logic.moves
from random_moves import run_random

In [59]:
dict_moves = {'up': 0, 'down': 1, 'left': 2, 'right': 3}

In [60]:
aa = tar_list[1][:5]
print(aa)

['down', 'up', 'right', 'right', 'up']


In [63]:
list(map(dict_moves.get, aa))

[1, 0, 3, 3, 0]

In [35]:
## Generate random sessions for initial training set

In [72]:
sess_points, sess_bests, sess_moves, mov_df, tar_list = run_random(n_sessions=1000, filed_size=4, save_df=True)

In [73]:
len(tar_list)

1000

In [74]:
len(tar_list)

1000

In [75]:
len(sess_points)

1000

In [76]:
np.max(sess_points)

3140.0

In [77]:
np.max(sess_bests)

256.0

In [78]:
np.max(sess_moves)

308

In [79]:
tar_list[1][:5]

['up', 'right', 'left', 'up', 'down']

In [80]:
mov_df[1].head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
3,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0
4,8.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
sess_points = np.array(sess_points)
print(sess_points[:10])
bests = sess_points.argsort()[-4:][::-1]
print(bests)

[1408.  760.  756.  960. 1136. 1212. 1280.  628. 1172. 2124.]
[363 919 848 249]


In [84]:
sess_points[919]

3112.0

In [101]:
def select_rand_sessions(n_sessions, rand_ratio):
    return random.sample(range(n_sessions), round(n_sessions * rand_ratio))

In [102]:
def select_elite_sessions(points_arr, best_ratio):
    return points_arr.argsort()[-round(len(points_arr) * best_ratio):][::-1]

In [107]:
elite_sessions = select_elite_sessions(sess_points, 0.1)
print(len(elite_sessions))
rand_sessions = select_rand_sessions(len(sess_points), 0)
print(len(rand_sessions))
sess_to_select = list(set(np.append(elite_sessions, rand_sessions)))
print(len(sess_to_select))
sess_to_select = list(map(int, sess_to_select))

100
0
100


In [108]:
Xtrain = pd.DataFrame()
ytrain = list()

In [109]:
for sess in sess_to_select:
    Xtrain = Xtrain.append(mov_df[sess], ignore_index=True)
    ytrain.append(tar_list[sess])

In [110]:
unlist = lambda list_of_lists: [y for x in list_of_lists for y in x]

In [111]:
ytrain = unlist(ytrain)
print(ytrain[:10])
ytrain = list(map(dict_moves.get, ytrain))
print(len(ytrain))
print(ytrain[:10])

['up', 'left', 'right', 'up', 'down', 'up', 'up', 'down', 'down', 'down']
22903
[0, 2, 3, 0, 1, 0, 0, 1, 1, 1]


In [112]:
len(Xtrain)

22903

In [121]:
np.random.seed(42)
num_class = 4

iter_num = 1

inputs = Input(shape=(16, ))
x = Dense(16, activation='linear')(inputs)
x = Dense(32, activation='linear')(inputs)
x = Dense(32, activation='linear')(inputs)
x = Dense(32, activation='linear')(inputs)
x = Dense(16, activation='linear')(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

checkpointer = ModelCheckpoint(str('../models_saved/' + 'iter' + str(iter_num) + '.hdf5'),
                               monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es= EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit([Xtrain],
                    y=to_categorical(ytrain), verbose=0,
                    batch_size=64,
                    validation_split=0.2,
                    shuffle=True, epochs=500,
                    callbacks=[checkpointer, es])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 16)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_53 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_54 (Dense)             (None, 4)                 68        
Total params: 1,140
Trainable params: 1,140
Non-trainable params: 0
_________________________________________________________________

Epoch 00001: val_acc improved from -inf to 0.26021, saving model to ../models_saved/iter1.hdf5

Epoch 00002: val_acc did not improve from 0.26021

Epoch 00003: val_acc did not improve from 0.26021

Epoch 00004: val_acc did not improve from 0.26021

Epoch 00005: val_acc d

In [ ]:
def get_predicted_field(model, n_sessions=100, filed_size=4, save_df=False): # finish me!!!!!!!!!
    sessions_points = []
    sessions_bests = []
    sessions_moves = []
    if save_df:
        moves_df = list()
        targets_list = list()
    for i in range(n_sessions):
        if save_df:
            sess_moves_df = pd.DataFrame(columns=range(filed_size ** 2))
            sess_targets_list = list()
        points = 0
        num_moves = 0
        field = get_initial_field(filed_size)
        while True:
            move = ['up', 'down', 'right', 'left'][randint(0, 3)]
            if save_df:
                sess_moves_df = sess_moves_df.append(pd.Series(field.flatten(), index=range(filed_size ** 2)),
                                                     ignore_index=True)
                sess_targets_list.append(move)
            field_moved, new_points = move_field(field, move)
            points += new_points
            num_moves += 1
            if not np.array_equal(field_moved, field):
                field = insert_new_cell(field_moved)
            if field_stucked(field):
                sessions_points.append(points)
                sessions_bests.append(np.max(field))
                sessions_moves.append(np.max(num_moves))
                if save_df:
                    moves_df.append(sess_moves_df)
                    targets_list.append(sess_targets_list)
                break
    if save_df:
        return sessions_points, sessions_bests, sessions_moves, moves_df, targets_list
    else:
        return sessions_points, sessions_bests, sessions_moves

In [153]:
field = get_initial_field(4)

In [155]:
field = np.array(field.flatten())
field.shape = (1, 16)
field

array([[0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

In [156]:
model.predict(field)

array([[0.24564777, 0.24676466, 0.25639886, 0.25118867]], dtype=float32)